In [1]:
import pandas as pd
from geopy.geocoders import Photon
from unidecode import unidecode
import unidecode

# RUTAS

### Justificación

Este notebook tiene como principal objetico el de organizar un data freme con las geo referencias geograficas  de las ciudades y puntos representativos que integran los corredores de migrantes y sus diferentes combinaciones.

La metodologia de trabajo consiste en:

1: Recopilar de diversas fuentes de información un listado con las ciudades que componen las rutas migratorias de acuerdo con autores principales en el tema que incluyen a  Rodolfo Casillas, CNDH y otros cuyas referencias se utilizan en la mayoria de los informes informativos y trabajos de investigacion sobre la Migracion en Transito Irregular en Mexico.

2: Preparar ese listado en CSV, que posteriormente sea re procesado para cumplir con los requerimientos de las herramientas de traceo de ArcGis.

3: Revisar tutoriales y definir requerimientos:

4: Adaptar CSV

5: Proyectar



In [2]:
# Cargar el archivo CSV
datos_csv = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/3. Rutas/Ciudades_Rutas.csv')


In [3]:
datos_csv.head(5)

,Ciudad,Referencia Geográfica,Descripción,Ruta,Latitud,Longitud,Fuente
0,"Puebla, Puebla",Puebla,NaN,Ruta del Pacifico,NaN,NaN,https://aleph.org.mx/cuales-son-las-principale...
1,"Tlaxcala, Tlaxcala",Tlaxcala,NaN,Ruta del Pacifico,NaN,NaN,"Casillas, 2008"
2,"Lechería, Estado de México",Lecheria,NaN,"Punto de inicio del viaje en ""La Bestia""",NaN,NaN,https://aleph.org.mx/cuales-son-las-principale...
3,"Tijuana, Baja California",Tijuana,NaN,Ruta del Pacifico,NaN,NaN,"Casillas, 2008"
4,"Mexicali, Baja California",Mexicali,NaN,NaN,NaN,NaN,"Casillas, 2009"


In [4]:
datos_csv.shape

(147, 7)

#### Encontrar y quitar repetidos

Columna: [Referencia Geográfica]

In [5]:
# Eliminar duplicados basados en la columna 'Referencia Geográfica'
datos_csv = datos_csv.drop_duplicates(subset='Referencia Geográfica', keep='first')

# Mostrar el DataFrame resultante para verificar los cambios
print(datos_csv)


                          Ciudad Referencia Geográfica  Descripción  \
0                 Puebla, Puebla                Puebla          NaN   
1             Tlaxcala, Tlaxcala              Tlaxcala          NaN   
2     Lechería, Estado de México              Lecheria          NaN   
3       Tijuana, Baja California               Tijuana          NaN   
4      Mexicali, Baja California              Mexicali          NaN   
..                           ...                   ...          ...   
142            Córdoba, Veracruz               Cordoba          NaN   
143              Ixtepec, Oaxaca               Ixtepec          NaN   
144  Tultitlán, Estado de México             Tultitlan          NaN   
145           Celaya, Guanajuato                Celaya          NaN   
146            Apizaco, Tlaxcala               Apizaco          NaN   

                                         Ruta  Latitud  Longitud  \
0                           Ruta del Pacifico      NaN       NaN   
1          

In [6]:
datos_csv.head()

,Ciudad,Referencia Geográfica,Descripción,Ruta,Latitud,Longitud,Fuente
0,"Puebla, Puebla",Puebla,NaN,Ruta del Pacifico,NaN,NaN,https://aleph.org.mx/cuales-son-las-principale...
1,"Tlaxcala, Tlaxcala",Tlaxcala,NaN,Ruta del Pacifico,NaN,NaN,"Casillas, 2008"
2,"Lechería, Estado de México",Lecheria,NaN,"Punto de inicio del viaje en ""La Bestia""",NaN,NaN,https://aleph.org.mx/cuales-son-las-principale...
3,"Tijuana, Baja California",Tijuana,NaN,Ruta del Pacifico,NaN,NaN,"Casillas, 2008"
4,"Mexicali, Baja California",Mexicali,NaN,NaN,NaN,NaN,"Casillas, 2009"


In [7]:
# Crear una nueva columna 'Estado' extrayendo el texto después de la coma
datos_csv['Estado'] = datos_csv['Ciudad'].apply(lambda x: x.split(', ')[1] if ', ' in x else '')


In [8]:
from unidecode import unidecode as quitar_acentos

# Ahora aplicar la función corregida a las columnas 'Ciudad' y 'Estado'
datos_csv['Ciudad'] = datos_csv['Ciudad'].apply(quitar_acentos)
datos_csv['Estado'] = datos_csv['Estado'].apply(lambda x: quitar_acentos(x) if pd.notnull(x) else x)


In [15]:
# Definir las listas de estados para cada ruta, asegurándose de que todas las entradas están correctamente formateadas como strings
estados_ruta_golfo = ['Tabasco', 'Nuevo Leon', 'Veracruz', 'Tamaulipas']
estados_ruta_pacifico = ['Guerrero', 'Michoacan', 'Jalisco', 'Nayarit', 'Sinaloa', 'Sonora', 'Baja California', 'Baja California Sur','Colima']
estados_ruta_centro = ['San Luis Potosi','Estado de Mexico', 'Tlaxcala', 'Ciudad de Mexico', 'Puebla', 'Morelos', 'Hidalgo', 'Zacatecas', 'Durango', 'Chihuahua', 'Coahuila', 'Queretaro', 'Aguascalientes', 'Guanajuato']
ruta_sur = ['Chiapas', 'Oaxaca', 'Tabasco', 'Campeche', 'Quintana Roo', 'Yucatan']

# Actualizar la función para asignar la ruta basada en el estado
def asignar_ruta(estado):
    estado = estado.strip()  # Eliminar espacios en blanco al principio y al final
    if estado in estados_ruta_golfo:
        return 'Ruta del Golfo'
    elif estado in estados_ruta_pacifico:
        return 'Ruta del Pacífico'
    elif estado in estados_ruta_centro:
        return 'Ruta Centro'
    elif estado in ruta_sur:
        return 'Ruta Sur'
    else:
        return 'Ruta no identificada'  # Para estados que no coinciden con las listas

# Aplicar la función al DataFrame para crear una nueva columna 'Ruta'
datos_csv['Ruta'] = datos_csv['Estado'].apply(asignar_ruta)

In [16]:
# Reordenar las columnas para colocar 'Estado' después de 'Ciudad'
column_order = ['Ciudad', 'Estado', 'Referencia Geográfica', 'Descripción', 'Ruta', 'Latitud', 'Longitud', 'Fuente']

# Aplicar el nuevo orden de columnas al DataFrame
datos_csv = datos_csv[column_order]

### PARTE 2: Geolocate Addresses

In [17]:
# Inicializar el geolocalizador de Photon
geolocator = Photon(user_agent="measurements", timeout=10)

# Definir la función para obtener latitud y longitud
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address)
        if location:  # Verificar si se encontró la ubicación
            return location.latitude, location.longitude
        else:
            return '', ''  # Devolver cadenas vacías si no se encuentra la ubicación
    except AttributeError:
        return '', ''

# Aplicar la función a la columna 'Ciudad' y dividir los resultados en las columnas 'Latitud' y 'Longitud'
datos_csv[['Latitud', 'Longitud']] = datos_csv['Ciudad'].apply(lambda x: pd.Series(get_lat_lon(x)))

In [18]:
datos_csv.head(60)

,Ciudad,Estado,Referencia Geográfica,Descripción,Ruta,Latitud,Longitud,Fuente
0,"Puebla, Puebla",Puebla,Puebla,NaN,Ruta Centro,18.833333,-98.000000,https://aleph.org.mx/cuales-son-las-principale...
1,"Tlaxcala, Tlaxcala",Tlaxcala,Tlaxcala,NaN,Ruta Centro,19.416667,-98.166667,"Casillas, 2008"
2,"Lecheria, Estado de Mexico",Estado de Mexico,Lecheria,NaN,Ruta Centro,19.602439,-99.183295,https://aleph.org.mx/cuales-son-las-principale...
3,"Tijuana, Baja California",Baja California,Tijuana,NaN,Ruta del Pacífico,32.531740,-117.019529,"Casillas, 2008"
4,"Mexicali, Baja California",Baja California,Mexicali,NaN,Ruta del Pacífico,32.640525,-115.474899,"Casillas, 2009"
5,"San Luis Rio Colorado, Sonora",Sonora,San Luis Río Colorado,NaN,Ruta del Pacífico,32.451796,-114.765254,"Casillas, 2010"
6,"Sonoyta, Sonora",Sonora,Sonoyta,NaN,Ruta del Pacífico,31.863444,-112.849739,"Casillas, 2011"
7,"Nogales, Sonora",Sonora,Nogales,NaN,Ruta del Pacífico,31.328506,-110.945047,"Casillas, 2012"
8,"Agua Prieta, Sonora",Sonora,Agua Prieta,NaN,Ruta del Pacífico,31.333015,-109.559427,"Casillas, 2013"
9,"Ciudad Juarez, Chihuahua",Chihuahua,Ciudad Juárez,NaN,Ruta Centro,31.737257,-106.485655,"Casillas, 2014"


In [19]:
# Ruta original del archivo
ruta_original = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/3. Rutas/Ciudades_Rutas.csv'

# Modificar el nombre del archivo para incluir '_Final'
ruta_final = ruta_original.replace('Ciudades_Rutas.csv', 'Ciudades_Rutas_Final.csv')

# Guardar el DataFrame actualizado en el nuevo archivo
datos_csv.to_csv(ruta_final, index=False)

print(f"Archivo actualizado guardado en: {ruta_final}")


Archivo actualizado guardado en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/3. Rutas/Ciudades_Rutas_Final.csv
